# Muestreo estratificado
    - Un estrato es un subconjunto de la población. La suma de las unidades en todos los estratos es la población
    
    - Se muestrea cada estrato por separado, se hacen las estimaciones para cada uno y luego se combinan para obtener los estimadores de la población
    
    - La estimación por estratos se emplea cuando existe una población heterogenea y existe una preconcepción de que existen grupos (estratos) que permiten reducir la varianza dentro de cada uno
   
    - El efecto de la formación de estratos es reducir la variabilidad de los estimadores
    
    - Se pueden tener diferentes diseños de muestra para cada estrato
    
    - Los estimadores por estrato no son relevantes ni estadísticamente adecuados, debido a que el objetivo final siempre será obtener los estimadores para la población

### Conclusiones
    - Con una semilla fija, al aumentar el tamaño de muestra en el método sample, un nuevo elemento es agregado, preservando los anteriores. Por ejemplo, cuando se tiene n=1, la unidad [x] es seleccionada; cuando n=2, las unidades [x, y]; cuando n=3, [x, y,z ] y sucesivamente
    
    - Notese que los estimadores al usar estrato son una muestra ponderada entre el número de unidades en el estrato respecto al total de undiades en la población
    
    - Definir los estratos es un paso sensible, debido a qué idealmente la varianza dentro de cada uno se debe reducir, por lo que escoger estratos arbitrariamente, no forzosamente mejorará los estimadores

### 0] Ejemplo
    - Se observa que existen dos grupos que si se consideran por separado tendrían varianza reducida entre ellos, pero la varianza total aumenta al considerarlos todos al mismo tiempo

In [88]:
import numpy as np

# Supose a population
P = np.array([1, 2, 1, 1, 2, 3, 1, 2, 1, 2,
              7, 7, 9, 8, 7, 7, 7, 8, 8, 9])

# Results
print("La varianza poblacional es de: ", round(P.var(axis=0),2))
print("La varianza del primer estrato es: ", round(P[:10].var(axis=0),2))
print("La varianza del segundo estrato es: ", round(P[10:].var(axis=0),2))

La varianza poblacional es de:  9.83
La varianza del primer estrato es:  0.44
La varianza del segundo estrato es:  0.61


### 1] Definir una población
    - Notación:
        L: Número de estratos
        N: Tamaño poblacional
        N_h: Unidades poblacionales dentro de cada estrato
        Y[h,i]: Observación en el estrato h de la unidad i
    
    - Los parámetros como la media, varianza y total se calculan por separado para cada estrato como si fueran una población por separado. Para los cálculos poblacionales, los elementos son considerados con el mismo peso de importancia.  

In [1]:
import pandas as pd
import numpy as np

# Define the strata
# ... 600 is the total population, while the other elements in the list
# ... represent the units within each strata
# ... sample size is initially defined only for m.a.s.
L = 3
N = [600, 300, 200, 100]
n = [60, 0, 0, 0]
confidence = 0.95

# Create DF outlining the strata for each unit
strata = np.repeat([1,2,3], N[1:])
df_population = pd.DataFrame(strata, columns=["strata"])

# Define score variable
score =                  np.random.normal(loc=6,   scale=1.5, size=N[1])
score = np.append(score, np.random.normal(loc=7.5, scale=1.0, size=N[2]))
score = np.append(score, np.random.normal(loc=8.5, scale=0.7, size=N[3]))
score = np.where(score>10, 10, score)
score = np.where(score< 0,  0, score)
df_population["score"] = score

# Define money variable
money =                  np.random.normal(loc=1500, scale=300, size=N[1])
money = np.append(money, np.random.normal(loc=1000, scale=200, size=N[2]))
money = np.append(money, np.random.normal(loc= 800, scale=100, size=N[3]))
money = np.where(money<0, 0, money)
df_population["money"] = money

# Define sex variable
sex =                (0.50 < np.random.uniform(size=N[1]))
sex = np.append(sex, (0.25 < np.random.uniform(size=N[2])))
sex = np.append(sex, (0.75 < np.random.uniform(size=N[3])))
df_population["sex"] = sex.astype(int)

# Output
print("Parámetros poblacionales\n")
print("La media de calificación es: ", round(df_population["score"].mean(axis=0),2))
print("El total de dinero es: ", int(df_population["money"].sum(axis=0)))
print("La proporción hombres/mujeres es: ", round(df_population["sex"].mean(axis=0),2))


Parámetros poblacionales

La media de calificación es:  6.89
El total de dinero es:  730446
La proporción hombres/mujeres es:  0.57


### 2] Estimaciones con m.a.s. no estratificado
    - Se define una muestra aleatoria sin considerar los estratos y se calculan los estimadores de las variables

In [90]:
from scipy.stats import t

# Not stratified sampling means equal probability of being sampled
sampling_w = np.repeat(1/n[0], N[0])

# Sample population dataframe
# ... replace - sets if sampling is with or withoun replace
# ... ignore_index - if true returns original index, else creates new indexation
# ... random_state - placeholder for random seedx
# ... weights - Pass a coupled series with the probability of being in sample for each unit
df_sample = df_population.sample(n=n[0], replace=False, ignore_index=False, random_state=432, weights=sampling_w)
df_sample["score"].mean(axis=0)



# Score --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
mean = df_sample["score"].mean(axis=0)
var = df_sample["score"].var(axis=0, ddof=1)
mean_std = ( (1-n[0]/N[0]) * var/n[0] )**0.5
mean_interval = mean + mean_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))



# Money  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
total = N[0] * df_sample["money"].mean(axis=0)
var = df_sample["money"].var(axis=0, ddof=1)
total_std = N[0] * ( (1-n[0]/N[0]) * var/n[0] )**0.5
total_interval = total + total_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))



# Sex --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
prop = df_sample["sex"].mean(axis=0)
prop_std = ( (1-n[0]/N[0]) * N[0]/(N[0]-1) * 1/n[0] * prop*(1-prop) )**0.5
prop_interval = prop + prop_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))



# Results --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
print("Bajo m.a.s. (sin emplear estratos) se estima que\n")
print("La media de calificación es: ", round(mean,2), " | ", np.round(mean_interval,2))
print("El total de dinero es: ", int(total), " | ", total_interval.astype(int))
print("La proporción hombres/mujeres: ", round(prop,2), " | ", np.round(prop_interval,2))

Bajo m.a.s. (sin emplear estratos) se estima que

La media de calificación es:  7.33  |  [7.03 7.64]
El total de dinero es:  687443  |  [636473 738413]
La proporción hombres/mujeres:  0.53  |  [0.41 0.66]


### 3] Estimaciones con estratos
    - En general, los estimadores poblacionales utilizando estratos es una ponderación de los estimadores para cada estrato, por la proporción de población que engloba cada uno

    - Aunque el diseño de muestra no haya sido establecido como estratificado desde un principio, la estimación por estratos permite mejorar la calidad de los estimadores

    - MEDIA
        - La media poblacional se estima como:

            mean_0 = sum( W_h * mean_h   for h in range(1,L+1))
                donde:
                mean_p: Media poblacional
                W_h = N_h/N_0
                mean_h: Media muestral para el estrato
                range(L): Recorrer los estratos 1, ..., L

        - La varianza del estimador se calcula como:
        
            Var(mean_p) = sum( W_h^2 * (1-n_h/N_h) * s_h^2/n_h   for j in range(1,L+1))
                donde:
                Var(mean_p): Varianza del estimador de la media
                W_h: N_h/N_0
                s_h^2: varianza observada dentro del estrato
            
    - TOTAL
        - El total poblacional se estima como:

            total_0 = sum(total_h   for h in range(1,L+1))
                donde:
                total_0: Total poblacional
                total_h: Total del estrato = N_h*mean_h
                N_h: Tamaño poblacional del estrato
                mean_h: Media observada en el estrato
                
        - La varianza del estimador se calcula como:
        
            Var(total) = sum(N_h^2 * (1-n_h/N_h) * 1/n_h * s_h^2   for h in range(1,L+1))
                donde:
                Var(total): Varianza del estimador del total
                N_h y n_h: Tamaño poblacional y muestral del estrato
                s_h^2: Varianza observada dentro del estrato
            
            
    - PROPORCIÓN
        - La proporción poblacional se estima como:
            
            prop_0 = sum( W_h * prop_h   for h in range(1,L+1))
                donde:
                prop_0: Proporcion poblacional
                W_h: N_h/N_0
                prop_h: Proporción observada en el estrato
                
        - La varianza del estimador se calcula como:
            Var(prop_0) = sum( W_h^2 * (1-n_h/N_h) * 1/(n_h-1) * prop_h*(1-prop_h)   for h in range(1.L+1))
                donde:
                Var(prop_0): Varianza del estimador del total
                W_h = N_h/N_0
                prop_h: Proporción observada en el estrato

In [91]:
# Define series with strata label
strata = df_sample["strata"]

# Get sample size for each strata 
for h in range(1, L+1):
    n[h] = (strata == h).sum(axis=0)
    

    
# Score --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---    
# Calculate estimation of mean
mean = [0, 0, 0 , 0]
# Loop in each strata (1, 2, 3)
for h in range(1, L+1):
    mean[h] = df_sample["score"][strata==h].mean(axis=0)
    mean[0] += N[h]/N[0] * mean[h]
    
# Variance of estimator
mean_var = 0
for h in range(1, L+1):
    mean_var += (N[h]/N[0])**2 * (1 - n[h]/N[h]) * 1/n[h] * df_sample["score"][strata==h].var(axis=0, ddof=1)
mean_std = mean_var**0.5

# Confidence interval
mean_interval = mean[0] + mean_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))
    
    
    
# Money --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# Calculate estimation of total
total = [0, 0, 0, 0]
# Loop in each strata (1, 2, 3)
for h in range(1, L+1):
    total[h] = N[h] * df_sample["money"][df_sample["strata"]==h].mean(axis=0)
    total[0] += total[h]

# Variance of estimator
total_var = 0
# Loop in each strata (1, 2, 3)
for h in range(1, L+1):
    total_var += N[h]**2 * (1 - n[h]/N[h]) * 1/n[h] * df_sample["money"][strata==h].var(axis=0, ddof=1)
total_std = total_var**0.5
    
# Confidence interval
total_interval = total[0] + total_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))



# Sex --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# Calculate estimation of proportion
prop = [0, 0, 0, 0]
# Loop in each strata (1, 2, 3)
for h in range(1, L+1):
    prop[h] = df_sample["sex"][strata==h].mean(axis=0)
    prop[0] += N[h]/N[0] * prop[h]
    
# Calculate variance of estimator
prop_var = 0
for h in range(1, L+1):
    prop_var += (N[h]/N[0])**2 * (1 - n[h]/N[h]) * 1/(n[h]-1) * (prop[h]*(1-prop[h]))
prop_std = prop_var**0.5

# Confidence interval
prop_interval = prop[0] + prop_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))



# # Results --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
print("Utilizando estratos se estima que\n")
print("La media de calificación es: ", round(mean[0],2), " | ", np.round(mean_interval,2))
print("El total de dinero es: ", int(total[0]), " | ", total_interval.astype(int))
print("La proporción hombres/mujeres: ", round(prop[0],2), " | ", np.round(prop_interval,2))

Utilizando estratos se estima que

La media de calificación es:  7.1  |  [6.85 7.34]
El total de dinero es:  734762  |  [699506 770019]
La proporción hombres/mujeres:  0.54  |  [0.41 0.67]
